In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score
np.random.seed(42)

# Data Exploration

In [2]:
index_path = './trec06p/label/index'
data_dir = './trec06p/data/'
with open(index_path) as f:
    index_list = f.readlines()

index_dict = {}
for index in index_list:
    index = index.split()
    index_dict[index[1]] = 1 if index[0] == 'spam' else 0

corpus = []
labels = []
for key, value in index_dict.items():
    with open(os.path.join(data_dir, key)) as f:
        try:
            corpus.append(f.read())
            labels.append(value)
        except:
            pass

raw_data = pd.DataFrame({'label': labels, 'text': corpus})
raw_data.head()

,label,text
0,0,Received: from rodan.UU.NET by aramis.rutgers....
1,1,Received: from unknown (HELO groucho.cs.psu.ed...
2,1,Received:\n\tfrom 24-151-178-89.dhcp.kgpt.tn.c...
3,0,Received: from psuvax1.cs.psu.edu ([130.203.2....
4,1,Received: from 201-1-198-159.dsl.telesp.net.br...


### Toy dataset (没什么用，拿来测试模型能不能正常跑)

In [3]:
toy = [
    {"text": "Win big prizes now!", "label": "spam"},
    {"text": "Important information about your account", "label": "spam"},
    {"text": "Dear friend, I have a business proposal for you", "label": "spam"},
    {"text": "This is a normal email, nothing suspicious", "label": "not spam"},
    {"text": "Another normal message", "label": "not spam"},
    {"text": "Limited time offer, buy now and save big!", "label": "spam"}
]
toy_df = pd.DataFrame(toy, columns=["label", "text"])
toy_df['label'] = toy_df['label'].apply(lambda x: 1 if x == 'spam' else 0)
# data = toy_df # uncomment this line to use the toy dataset

# Doing things to the dataset

### 1.1 Converting to lower case

In [4]:
data = raw_data.copy()
data['text'] = data['text'].apply(lambda x: x.lower())

### 1.2 Data balancing （发现没有balance效果更好)

In [5]:
# data = data.groupby('label').apply(lambda x: x.sample(n=10000, replace=True)).reset_index(drop=True)
data.label.value_counts()

1    20030
0    12371
Name: label, dtype: int64

### 2. Train test split

In [6]:
def train_test_split(data: pd.DataFrame, test_size=0.3):
    train_df = data.sample(frac=1-test_size)
    test_df = data.drop(train_df.index)
    return train_df.reset_index(drop=True), test_df.reset_index(drop=True)

train_df, test_df = train_test_split(data, test_size=0.3)

# Feature Engineering

"earn money,” “act now,” “click here,” “buy now,” “limited time offer,” “get rich quick,” “earn extra cash,” “make money fast,” “guaranteed,” “winner,” “bonus,” and “urgent.

### 1. Feature extraction functions
Todo: use count vectorizer to see if it improves

In [7]:
def get_features_bernoulli(text: str):
    keywords = [
        'FREE', 'free', 'earn money', 'act now', 'click here', 'buy now',
        'limited time offer', 'get rich quick', 'earn extra cash', 'make money fast',
        'guaranteed', 'winner', 'bonus', 'urgent', 'credit card', 'lowest price',
        'amazing', 'incredible deal', 'no cost', 'risk free', 'special promotion',
        'exclusive offer', 'million dollars', 'once in a lifetime', 'password',
        'account suspended', 'confidentiality', 'discount',
        'win', 'winner', 'cash', 'prize', 'exclusive', 'urgent', 'important', 
        'free', 'act now', 'offer', 'credit', 'cheap', 'bonus', 'click', 'apply', 'buy', 'limited', 'guaranteed', 'save'
    ]

    features = {f'has_{keyword.replace(" ", "_")}': text.lower().count(keyword.lower()) > 0 for keyword in keywords}
    return features

def get_features_multinomial(text: str):
    keywords = [
        'FREE', 'free', 'earn money', 'act now', 'click here', 'buy now',
        'limited time offer', 'get rich quick', 'earn extra cash', 'make money fast',
        'guaranteed', 'winner', 'bonus', 'urgent', 'credit card', 'lowest price',
        'amazing', 'incredible deal', 'no cost', 'risk free', 'special promotion',
        'exclusive offer', 'million dollars', 'once in a lifetime', 'password',
        'account suspended', 'confidentiality', 'discount',
        'win', 'winner', 'cash', 'prize', 'exclusive', 'urgent', 'important', 
        'free', 'act now', 'offer', 'credit', 'cheap', 'bonus', 'click', 'apply', 'buy', 'limited', 'guaranteed', 'save'
    ]

    features = {f'count_{keyword.replace(" ", "_")}': text.lower().count(keyword.lower()) for keyword in keywords}
    return features


# Pre-processing

In [8]:
def preprocess(X):
    features = X.iloc[:,0].apply(get_features_multinomial).apply(pd.Series)
    X = pd.concat([X, features], axis=1).drop('text', axis=1)
    return X

X_train = preprocess(train_df.drop('label', axis=1))
X_test = preprocess(test_df.drop('label', axis=1))
y_train = train_df['label']
y_test = test_df['label']
X_train.head()

,count_FREE,count_free,count_earn_money,count_act_now,count_click_here,count_buy_now,count_limited_time_offer,count_get_rich_quick,count_earn_extra_cash,count_make_money_fast,...,count_exclusive,count_important,count_offer,count_credit,count_cheap,count_click,count_apply,count_buy,count_limited,count_save
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Using sklearn vectorize instead for feature engineering

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2

vectorizer = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.7)

def preprocess(X, vectorizer):
    features = vectorizer.transform(X['text'])
    return features

# Fit the vectorizer on the training data only
vectorizer.fit(train_df['text'])

X_train_tfidf = preprocess(train_df, vectorizer)
X_test_tfidf = preprocess(test_df, vectorizer)
y_train_tfidf = train_df['label']
y_test_tfidf = test_df['label']

# Modelling

### 1. MultinomialNB with Tfidf

In [10]:
from naive_bayes import MultinomialNB

multi = MultinomialNB()
multi.fit(X_train_tfidf, y_train_tfidf)
y_test_multi_tfidf = multi.predict(X_test_tfidf)
accuracy_score(y_test_multi_tfidf, y_test)

0.9265432098765433

### 1.2 MultinomialNB with no Tfidf and just some keywords

In [12]:
from naive_bayes import MultinomialNB

multi = MultinomialNB()
model = multi.fit(X_train, y_train)
y_test_multi = multi.predict(X_test)
accuracy_score(y_test_multi, y_test)

0.6309670781893004

### 2. BernoulliNB

In [ ]:
from naive_bayes import BernoulliNB
# from baseline import BernoulliNB

bern = BernoulliNB()
bern.fit(X_train, y_train)
y_pred_bern = bern.predict(X_test)
accuracy_score(y_test, y_pred_bern)

### 3. Sklearn baseline

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB as sk_BernoulliNB
from sklearn.model_selection import train_test_split

bern_sk = sk_BernoulliNB()
bern_sk.fit(X_train, y_train)
y_pred_sk = bern_sk.predict(X_test)
accuracy_score(y_test, y_pred_sk)

# Question 1: 5%, 50%, 100% of dataset for training

In [ ]:
for size in [0.05, 0.5, 1]:
    print("Test size:", size)
    train_df, test_df = train_test_split(data, test_size=size)
    X_train = preprocess(train_df.drop('label', axis=1))
    X_test = preprocess(test_df.drop('label', axis=1))
    y_train = train_df['label']
    y_test = test_df['label']
    bern = BernoulliNB()
    bern.fit(X_train, y_train)
    y_pred_bern = bern.predict(X_test)
    print(f"Custom BernoulliNB accuracy for test size {size}: {accuracy_score(y_test, y_pred_bern)}")

    multi = MultinomialNB()
    multi.fit(X_train, y_train)
    y_test_multi = multi.predict(X_test)
    print(f"Custom MultinomialNB accuracy for test size {size}: {accuracy_score(y_test_multi, y_test)}")

    print("\n")
    

# K-Fold cross validation

In [ ]:
from sklearn.model_selection import KFold

k = 5

kf = KFold(n_splits=k, shuffle=True, random_state=42)

for train_index, test_index in kf.split(data):
    train_df = data.iloc[train_index]
    test_df = data.iloc[test_index]

    X_train = preprocess(train_df.drop('label', axis=1))
    X_test = preprocess(test_df.drop('label', axis=1))
    y_train = train_df['label']
    y_test = test_df['label']

    myBNB = BernoulliNB()
    myBNB.fit(X_train, y_train)
    y_pred = myBNB.predict(X_test)
    print("Bernoulli NB:", (y_test == y_pred).mean())

    bnb = sk_BernoulliNB()
    bnb.fit(X_train, y_train)
    sk_y_pred = bnb.predict(X_test)
    print("Multinomial NB:", accuracy_score(y_test, sk_y_pred))

    print("\n")